## Import des librairies

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import from_json
from pymongo import MongoClient
import pprint as p
import pandas, json
from whatthelang import WhatTheLang
wtl = WhatTheLang()


## Réception des données, transformation, prédiction et enregistrement sur MongoDB


In [4]:
# Création d'un spark context, d'un sql context et d'un streaming context
sc = SparkContext("local[2]","Commentaires AirBnB")
sqlContext = SQLContext(sc)
ssc = StreamingContext(sc, 1)
socket_stream = ssc.socketTextStream("localhost",9999)

# Récupération sur une fenêtre d'1s
flux = socket_stream.window( 1 )

# Traitement sur chaque ligne du commentaire

def TraitementCommentaire(x):
 # Récupération des RDD dans une liste
    x = x.collect()

 # Gestion de la fin du fichier
    if (x == 'Fini'):
        # arrêt des context
        global ssc
        global sqlContext
        global sc
        ssc.stop()
        sqlContext.stop()
        sc.stop()
        print("Le traitement du flux est terminé")
        return 1
    
    # Connection à la base MongoDB (qui tourne sur Docker)
    client = MongoClient('localhost:27017')
    db=client.test.Comments 

    # Gestion des différents item de la liste
    for item in x:
        comment_texte = item
        print(comment_texte)
        
        #transformation en dataframe pandas
        comment = pandas.read_json(comment_texte,typ='series').to_frame().transpose()
        comment['comments']=comment['comments'].replace("\r\n", "")
        comment['comments']=comment['comments'].replace("\n", "")
        comment['comments']=comment['comments'].astype(str)
        # Détection de la langue et ajout au dataframe
        comment['langage']=comment['comments'].apply(wtl.pred_prob)
        comment['langue']=comment['langage'].str[0].str[0].str[0]
        comment['lg_proba']=comment['langage'].str[0].str[0].str[1]
        comment=comment.drop(['langage'],axis=1)

        # insertion dans MongoDB si Français ou Anglais avec une probabilité à plus de 80%
        db.insert_many(comment[(comment['lg_proba']>0.8) & ((comment['langue']=='en') | (comment['langue']=='fr'))].to_dict('records'))
    
    # Fermeture du client MongoDB
    client.close()
    return 0

# Traitement de tous les RDD reçus dans le streaming en appliquant la fonction précédente
flux.foreachRDD(TraitementCommentaire)

# démarrage du flux
ssc.start()

{"listing_id":3109,"id":208779822,"date":1509667200000,"reviewer_id":4142888,"reviewer_name":"Patricia","comments":"Un petit nid fouiller douillet situé dans  appartement tranversant, très lumineux avec une fenêtre dans chaque piece! Situé au 4ème étage, sans ascenseur, même après une bonne journée de marche ne nous a pas paru pénalisant. Mais au contraire, dès l'entrée dans l'appartement,  une belle décoration, très confortable, bien équipé, un VRAI lit en 160cm digne de se nom vous, bous fera oublié les marchés montées....un vrai airbnb, où Anne nous confie son lieux de vie en toute confiance. \nNotre hôtes a été très réactive et nous a facilité notre séjour pour tous les points demandés.\nNous reviendrons chez Anne c'est sur si l'occasion se présente à nous.\n Merci encore."}
{"listing_id":3109,"id":234257457,"date":1518393600000,"reviewer_id":3539452,"reviewer_name":"Dominique","comments":"The host canceled this reservation 8 days before arrival. This is an automated posting."}
{"l

## Arrêt du streaming avant la fin du traitement du flux

In [5]:
ssc.stop()
sc.stop()

## Test des données présentes en base


In [6]:
# connection à la base MongoDB
client = MongoClient('localhost:27017')
db=client.test.Comments 


# récupération des données de la collection
data = db.find()

# display the data
for i in data:
    p.pprint(i)

{'_id': ObjectId('5b4318e2b726e80ec50b0c93'),
 'comments': 'Un petit nid fouiller douillet situé dans  appartement '
             'tranversant, très lumineux avec une fenêtre dans chaque piece! '
             'Situé au 4ème étage, sans ascenseur, même après une bonne '
             'journée de marche ne nous a pas paru pénalisant. Mais au '
             "contraire, dès l'entrée dans l'appartement,  une belle "
             'décoration, très confortable, bien équipé, un VRAI lit en 160cm '
             'digne de se nom vous, bous fera oublié les marchés montées....un '
             'vrai airbnb, où Anne nous confie son lieux de vie en toute '
             'confiance. \n'
             'Notre hôtes a été très réactive et nous a facilité notre séjour '
             'pour tous les points demandés.\n'
             "Nous reviendrons chez Anne c'est sur si l'occasion se présente à "
             'nous.\n'
             ' Merci encore.',
 'date': 1509667200000,
 'id': 208779822,
 'langue': 'fr',

Adapté des scripts du projet GitHub suivant : Stream-Processing-using-PySpark-and-MongoDB--master

https://github.com/vipulkrishnanmd/Stream-Processing-using-PySpark-and-MongoDB-